In [1]:
from aicspylibczi import CziFile
from timelapse_tools import daread
czi = CziFile("timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi")
img, dims = daread(czi)
print(dims)
img

timelapse_tools/tests/data/s_1_t_5_c_1_z_1.czi
STCZYX


,Array,Chunk
Bytes,5.77 MB,1.15 MB
Shape,"(1, 5, 1, 1, 624, 924)","(1, 1, 1, 1, 624, 924)"
Count,20 Tasks,5 Chunks
Type,uint16,numpy.ndarray


In [2]:
import dask.array as da
from typing import Callable, List

def percentile_norm(
    data: da.core.Array,
    min_p: float = 50.0,
    max_p: float = 99.8
) -> da.core.Array:
    # Get the norm by values
    norm_by = da.percentile(
        data.flatten(),
        [min_p, min_p]
    )

    # Norm
    normed = (data - norm_by[0]) / (norm_by[1] - norm_by[0])

    # Clip any values outside of 0 and 1
    clipped = da.clip(normed, 0, 1)

    # Scale them between 0 and 255
    return clipped * 255

def max_project(data: da.core.Array) -> da.core.Array:
    return da.max(data, axis=0)[0, :]

def process_channel(
    data: da.core.Array,
    dims: str,
    normalization_func: Callable = percentile_norm,
    projection_func: Callable = max_project
) -> List[da.core.Array]:
    frames = []
    # Iterate over frames and run the norm -> project
    for i in range(data.shape[dims.index("T")]):
        frames.append(
            projection_func(normalization_func(data))
        )
    
    return frames

In [3]:
channel_0 = process_channel(img[0, :, 0, :], "TZYX")
len(channel_0)

5

In [4]:
channel_0[0]

dask.array<getitem, shape=(624, 924), dtype=float64, chunksize=(624, 924), chunktype=numpy.ndarray>